In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv('train', names=['item'])
test_data = pd.read_csv('test', names=['item'])

In [3]:
df = pd.concat([train_data, test_data], ignore_index=True)
#беру не весь датасет так как ноутбук не вывозит
df = df[:500]

In [4]:
df.head()

,item
0,333396 267089 155959 353335 414000 339989 2741...
1,174197 335779 141676 119856 376664 311755 3881...
2,254861 213397 462645 403619 446211 221833 1572...
3,132371 199878 357751 92032 94315 435049 260359...
4,408028 326575 425054 152422 387296 382570 3505...


<h2>Преобразуем датафрейм к виду {user, item, order}</h2>

In [5]:
def preprocess_df(df_):
    df_['order'] = df_['item'].apply(lambda x: list(range(len(x.split(' '))-1, -1, -1)))
    df_ = df_.assign(liked_id=df_['item'].str.split(' '))
    df_ = df_.drop('item', axis=1)
    df_ = df_.rename(columns={"liked_id": "item"})
    df_ = df_.explode(['item', 'order'])
    df_ = df_.reset_index()
    df_ = df_.rename(columns={"index": "user"})    
    return df_

    

    

    
    

In [6]:
# res = preprocess_df(df)
# res.head()

### Датафрейм до препроцесса и после для 2х юзеров с больше чем 5 лайками

In [7]:
target_df = df[df.item.str.count(' ')>4]

In [9]:
random_sample = target_df.sample(2, random_state=1)

In [10]:
#до
random_sample

,item
304,429005 201532 341324 264817 199614 216592 3919...
340,115281 442075 201084 405374 192394 42914 10801...


In [11]:
# после
preprocess_df(random_sample)

,user,order,item
0,304,226,429005
1,304,225,201532
2,304,224,341324
3,304,223,264817
4,304,222,199614
...,...,...,...
354,340,4,268304
355,340,3,218871
356,340,2,80711
357,340,1,403945


## Homework

Исходные данные - Yandex Cup 2022 RecSys:
- Описание соревнования - https://contest.yandex.ru/yacup/contest/41618/problems/
- Данные - https://disk.yandex.ru/d/SI1aAooPn9i8TA
- Описание данных - в архиве likes_data.zip три файла:
  - train - обучающий набор данных. Каждая строка - последовательность id треков, которые лайкнул один пользователь. Гарантируется, что лайки даны в той последовательности, в которой их ставил пользователь.
  - test - набор тестовых данных. Имеет точно такой же формат, но в каждой строке не хватает последнего лайка, который надо предсказать.
  - track_artists.csv - информация о исполнителях треков. Гарантируется, что у каждого трека есть ровно один исполнитель. Для треков, у которых фактически несколько исполнителей, мы оставили того, который считается основным исполнителем трека.
- Описание сабмита - в качестве решения необходимо отправить файл, в котором для каждого пользователя из test в отдельной строке будет не более 100 треков, разделенных пробелом. Гарантируется, что у каждого пользователя будет только 1 лайк в тесте
- Метрика - MRR@100

Промежуточная задача - преобразовать данные в pandas.DataFrame вида {user, item, order}, где order - порядковый номер с конца (0 - самый "свежий" лайк, чем больше order, тем позже был поставлен лайк)

**Итоговая задача** - построить схему валидации для данного соревнования с учетом особенностей сорвенования
- Между `train` и `test` не должно быть общих пользователей
- Количество фолдов задается через параметр класса `n_folds`
- В `test` должно быть не более `p` последних треков (параметр класса `p`)

In [12]:
class UsersKFoldPOut():
    def __init__(self, n_folds, p, user_column, item_column, random_seed=23):
        self.n_folds = n_folds
        self.random_seed = random_seed
        self.user_column = user_column
        self.item_column = item_column
        self.p = p
    
    def split(self, df):
        users = df[self.user_column].unique()
        users_count = len(users)
        
        np.random.seed(self.random_seed)
        np.random.shuffle(users)
        
        fold_sizes = np.full(self.n_folds, users_count // self.n_folds, dtype=int)
        fold_sizes[: users_count % self.n_folds] += 1
        current = 0
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size
            test_fold_users = users[start:stop]
            test_mask = df['user'].isin(test_fold_users)
            train_mask = ~test_mask
            test_mask = (df['user'].isin(test_fold_users)) & (df.groupby('user').cumcount() < self.p)
            
            yield train_mask, test_mask

In [13]:
prepr_df = preprocess_df(df)
cv = UsersKFoldPOut(n_folds=3, p=3, user_column='user', item_column='item')

for i, (train_mask, test_mask) in enumerate(cv.split(prepr_df)):
    train = prepr_df[train_mask]
    test = prepr_df[test_mask]
    
    #Между train и test не должно быть общих пользователей
    assert len(set(train.user.unique()) & set(test.user.unique()))==0
    
    #В test должно быть не более p последних треков (параметр класса p)
    assert len(test['user']) - len(test['user'].drop_duplicates())<=test.user.nunique() * 3
    
    
    print(f'Fold#{i} | Train: {train.shape[0]}, Test: {test.shape[0]}')

Fold#0 | Train: 26284, Test: 501
Fold#1 | Train: 26284, Test: 501
Fold#2 | Train: 26291, Test: 498
